<a href="https://colab.research.google.com/github/Avnish3909/Algo-Tradering-Equal-weight-S-P-500/blob/main/Algo_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import json

In [ ]:
import os

stocks = pd.read_csv("sp_500_stocks.csv")

In [ ]:
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [ ]:
symbol = "AAPL"
api_url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey=SSZPRFBAD7SK0S8H"
print(api_url)
data = requests.get(api_url).json()
print(data)

https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=AAPL&apikey=SSZPRFBAD7SK0S8H
{'Global Quote': {'01. symbol': 'AAPL', '02. open': '185.0100', '03. high': '185.0400', '04. low': '182.2300', '05. price': '182.5200', '06. volume': '45119677', '07. latest trading day': '2024-02-23', '08. previous close': '184.3700', '09. change': '-1.8500', '10. change percent': '-1.0034%'}}


In [ ]:
price = data["Global Quote"]["05. price"]
marketcap = data["Global Quote"]["06. volume"]
print(price)
print(marketcap)

182.5200
45119677


In [ ]:
myColumns = [
    "Ticker",
    "Stock Price",
    "Market Capitalization",
    "Number of Shares to Buy",
]

In [ ]:
finalDataFrame = pd.DataFrame(columns=myColumns)

In [ ]:
seriesData = pd.Series([symbol, price, marketcap, "N/A"], index=myColumns)
seriesDataFrame = pd.DataFrame([seriesData], columns=myColumns)


In [ ]:
finalDataFrame = pd.concat([finalDataFrame, seriesDataFrame], ignore_index=True)

finalDataFrame

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,182.5200,45119677,N/A
1,AAPL,182.5200,45119677,N/A


In [ ]:
stocks[:10]

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


In [ ]:
data_frames = []
for x in stocks["Ticker"][:100]:
    blockurl = (
        f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={x}&apikey=SSZPRFBAD7SK0S8H"
    )
    print(blockurl)
    blockdata = requests.get(blockurl).json()
    temp_df = pd.DataFrame(
        [[x, blockdata["Global Quote"]["05. price"], blockdata["Global Quote"]["06. volume"], "N/A"]],
        columns=myColumns,
    )
    data_frames.append(
        temp_df
    )


https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=A&apikey=SSZPRFBAD7SK0S8H


KeyError: 'Global Quote'

In [ ]:
finalDataFrame = pd.concat(data_frames, ignore_index=True)
finalDataFrame

In [ ]:
portfolioSize = input("Please Enter The Value OF Your Portfolio:  ")
try:
    val = float(portfolioSize)
except ValueError:
    print("Input is not a number")
    portfolioSize = input("Please Enter The Value OF Your Portfolio again:  ")

positionSize = val / len(finalDataFrame.index)
for i in range(0, len(finalDataFrame.index)):
    finalDataFrame.loc[i, "Number of Shares to Buy"] = math.floor(
        positionSize / finalDataFrame.loc[i, "Stock Price"]
    )
print(finalDataFrame)

writer = pd.ExcelWriter("RecommendedTrades.xlsx", engine="xlsxwriter")
finalDataFrame.to_excel(writer, "RecommendedTrades", index=False)


In [ ]:
import xlsxwriter

backgroundColor = "#0a0a23"
fontColor = "#00ff00"

string_format = writer.book.add_format(
    {"font_color": fontColor, "bg_color": backgroundColor, "border": 1}
)
dollar_format = writer.book.add_format(
    {
        "num_format": "$0.00",
        "font_color": fontColor,
        "bg_color": backgroundColor,
        "border": 1,
    }
)
integer_format = writer.book.add_format(
    {
        "num_format": "0",
        "font_color": fontColor,
        "bg_color": backgroundColor,
        "border": 1,
    }
)


columnFormats = {
    "A": ["Ticker", string_format],
    "B": ["Stock Price", dollar_format],
    "C": ["Market Capitalization", dollar_format],
    "D": ["Number of Shares to Buy", integer_format],
}
for column in columnFormats.keys():
    writer.sheets["RecommendedTrades"].set_column(
        f"{column}:{column}", 20, columnFormats[column][1]
    )
writer.close()
print(finalDataFrame)
